A web crawler is set up to pull sale data off of p24. 
# SET THE CSV FILE NAME IN LAST CELL

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from tqdm import tqdm 

# Define the number of pages
num_iterations = 6

pricelist = []
sizelist = []

#tqdm(range(num_iterations), desc="Scraping Progress")

# Create a tqdm progress bar
for x in range(10):
    p24 = f"https://www.property24.com/apartments-for-sale/gauteng/1/p{x}?sp=pt%3d4000000%26fszt%3d150"
    req = requests.get(p24)
    soup = BeautifulSoup(req.text, "html.parser")

    for price in soup.find_all("span", "p24_price"):
        pricelist.append(price.text)   
    
    for size in soup.find_all("span", "p24_size"):
        sizelist.append(size.text)

pricelist = [price.strip().replace("\n", "").replace("\r", "").replace(" ", "").replace("R", "").replace("POA", "0") for price in pricelist]  
sizelist = [size.strip().replace("\n\n", "").replace("m²\n", "").replace("m² ", "").replace(" ha", "0").replace(" m²", "").replace(" ",  "").replace(".", "") for size in sizelist]  
csvdata = [[int(addr), int(price)] for addr, price in set(zip(pricelist, sizelist))]

print(f"CSVDATA: {len(csvdata), csvdata}")

CSVDATA: (201, [[640000, 53], [980850, 74], [999900, 40], [2199000, 121], [790000, 40], [950000, 71], [1350000, 83], [700000, 43], [450000, 78], [560000, 26], [599000, 97], [1985000, 77], [585000, 90], [1050000, 58], [1050000, 100], [299000, 81], [295000, 107], [750000, 62], [710000, 67], [1195000, 126], [775000, 44], [299000, 112], [580000, 82], [490000, 104], [1250000, 89], [1050000, 74], [680000, 55], [390000, 101], [784875, 109], [598000, 101], [800000, 66], [769990, 75], [300000, 104], [500000, 93], [1620000, 65], [1050000, 34], [859900, 96], [410000, 64], [360000, 83], [1950000, 72], [665000, 83], [1700000, 60], [761000, 56], [1985000, 84], [655000, 88], [640000, 83], [750000, 68], [650000, 68], [899000, 107], [395000, 78], [785000, 82], [410000, 78], [1680000, 108], [899000, 91], [6200, 79], [499000, 68], [320000, 65], [899000, 72], [1059000, 90], [1400000, 65], [761000, 50], [599000, 64], [1041780, 95], [750000, 110], [1000000, 61], [1050000, 131], [570000, 74], [1200000, 62], 

Using the propattr class the median and sd are calculated for the data. 

The data should be sorted by area name to avoid over use of the api. 

The data is sorted by name and pulled into its own list to find the co-ordinates using Bings Map Api.

In [14]:
import geocoder
from tqdm import tqdm

commonlocations = []


for data in csvdata:
    if data[1] not in commonlocations:
        commonlocations.append(data[1])

print(f"SIZE: {len(commonlocations)} {commonlocations}")

common_coords = []

failed = []

for addr in tqdm(commonlocations, desc="Finding Coordinates"):
    try:
        g = geocoder.bing(f'{addr + "northern cape" + "South Africa"}', key="YOURKEY")
        
        # Check if the geocoding request was successful
        if g.ok:
            results = g.json
            common_coords.append([results["lat"], results["lng"], addr])
        else:
            # Handle the case where the geocoding request failed
            print(f"Geocoding request failed for address: {addr}")
            failed.append(addr)
    except Exception as e:
        # Handle any exceptions that may occur during the geocoding request
        print(f"An error occurred while geocoding address: {addr}")
        print(f"Error details: {str(e)}")

SIZE: 19 [113, 72, 70, 57, 42, 101, 132, 55, 100, 121, 107, 84, 40, 65, 68, 46, 83, 137, 92]


Finding Coordinates: 100%|██████████| 19/19 [00:00<00:00, 19051.34it/s]

An error occurred while geocoding address: 113
Error details: unsupported operand type(s) for +: 'int' and 'str'
An error occurred while geocoding address: 72
Error details: unsupported operand type(s) for +: 'int' and 'str'
An error occurred while geocoding address: 70
Error details: unsupported operand type(s) for +: 'int' and 'str'
An error occurred while geocoding address: 57
Error details: unsupported operand type(s) for +: 'int' and 'str'
An error occurred while geocoding address: 42
Error details: unsupported operand type(s) for +: 'int' and 'str'
An error occurred while geocoding address: 101
Error details: unsupported operand type(s) for +: 'int' and 'str'
An error occurred while geocoding address: 132
Error details: unsupported operand type(s) for +: 'int' and 'str'
An error occurred while geocoding address: 55
Error details: unsupported operand type(s) for +: 'int' and 'str'
An error occurred while geocoding address: 100
Error details: unsupported operand type(s) for +: 'int

Enter data into csv for locations, price and coords.

In [2]:
import os
import csv

# Specify the target directory path
target_directory = r'C:\GitHub\Land-value-GIS\backups'

# Create the target directory if it doesn't exist
#if not os.path.exists(target_directory):
#    os.makedirs(target_directory)

# Define the file paths within the "backups" folder
csvdata_file_path = os.path.join(target_directory, "house_price_and_area.csv")
common_coords_file_path = os.path.join(target_directory, "northern_cape_area_coords.csv")

# Write to "prop_gauteng_locations_price.csv"
with open(csvdata_file_path, "w") as r:
    writer = csv.writer(r)
    writer.writerows(csvdata)

# Write to "gauteng_area_coords.csv"
#with open(common_coords_file_path, "w") as w:
#    writer = csv.writer(w)
#    writer.writerows(common_coords)


Move the files into the backups dir